<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/qwen50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-1.5B")
pipe(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I am a helpful assistant designed to assist with various tasks. What can I help you with today?.IsAny\n.IsAny\nI am a helpful assistant designed to assist with various tasks. What can I help you with today?.IsAny\n.IsAny\nI am a helpful assistant designed to assist with various tasks. What can I help you with today?.IsAny\n.IsAny\nI am a helpful assistant designed to assist with various tasks. What can I help you with today?.IsAny\n.IsAny\nI am a helpful assistant designed to assist with various tasks. What can I help you with today?.IsAny\n.IsAny\nI am a helpful assistant designed to assist with various tasks. What can I help you with today?.IsAny\n.IsAny\nI am a helpful assistant designed to assist with various tasks. What can I help you with today?.IsAny\n.IsAny\nI am a helpful assistant designed to assist with various tasks. What can I help you with today?.IsAny\n.IsAny\nI am 

In [2]:
import json
import pandas as pd
from google.colab import files
# Initialize the pipeline once
generator = pipeline("text-generation", model="Qwen/Qwen2.5-1.5B")

def generate_answer(question, num_samples):
    """
    Generates multiple answers for a given question using the model pipeline.
    """
    # Format the prompt
    prompt = f"أجب على السؤال التالي بجملة واحدة موجزة ولكن كاملة.\nQuestion: {question}\nAnswer:"

    # Generate the first 9 answers with temperature=1.0
    outputs_high_temp = generator(
        prompt,
        max_new_tokens=100,
        truncation=True,
        temperature=1.0,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=num_samples - 1
    )

    # Generate the last answer
    outputs_low_temp = generator(
        prompt,
        max_new_tokens=100,
        truncation=True,
        temperature=0.1,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=1
    )

    # Combine the results
    responses = [
        output['generated_text'].split("Answer:")[-1].strip() for output in outputs_high_temp
    ] + [
        outputs_low_temp[0]['generated_text'].split("Answer:")[-1].strip()
    ]

    return responses



Device set to use cuda:0


In [3]:



def main():
    """
    Main function to upload the dataset, process it, and generate answers.
    """
    from google.colab import drive, files
    import json
    import pandas as pd
    from IPython.display import FileLink
    drive.mount('/content/drive')
    # Replace with the path to your dataset on Google Drive
    file_path = f'/content/drive/MyDrive/test-open.jsonl'

    # Step 1: Upload the file
    #uploaded = files.upload()

    # Step 2: Read the uploaded file
    test_set = file_path
    data = []
    with open(test_set, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))

    # Step 3: Convert to DataFrame
    df = pd.DataFrame(data)

    # Initialize an empty list to store the results
    results = []

    # Step 4: Iterate over the dataset and generate answers
    for index, row in df.sample(n=50).iterrows():
        question = row['question']
        original_answer = row.get('answer', "")  # Use empty string if no original answer provided
        question_id = row.get('question_id', index)  # Use index if no question_id provided

        # Generate 10 answers for the 50 questions
        generated_answers = generate_answer(question=question, num_samples=10)



        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers
        })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)
    filename = "generated_answers.json"
    # Step 3: Save results to a JSON file
    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    display(FileLink(filename))
    files.download("generated_answers.json")

    # Run the main function
if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


   question_id                                           question  \
0      1190890  من كان يقود القوات الرومانية في معركة مونس غرا...   
1      1116911       ما هي الاستخدامات الشائعة لمقياس رينجلمان؟\n   
2      1173298                           متي ولد وليامز ريفيروس ؟   
3       960262  ما هي المساحة التي تغطيها الجبال في محافظة ناغ...   
4      1183267            ما هو VBX وما علاقته بـ Visual Basic؟\n   
5      1175973  ما هي العلامات والأعراض التي يعاني منها الأطفا...   
6      1069350                كم عدد الجذور القرآنية الكاملة؟\n\n   
7       960389                                      ما هي بابل؟\n   
8      1083028                        ما هي أغنية الخاتمة للأنمي؟   
9      1123968  من هو اللاعب الذي تغلب عليه لوكا بريسيل في نها...   
10      991898                            من أين جاء الماتابيون ؟   
11      952941  ما هي المدة الزمنية التي عاشها آخر سلف مشترك ل...   
12     1167674                                متي ولد بول هامشير؟   
13     1158950  ما هو جنس الفطر ال

/content/generated_answers.json

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>